<a href="https://colab.research.google.com/github/MuajiiTsai/110-1-NTU-DBME5028/blob/raw_label/%E8%82%87%E5%85%83%E5%88%A5%E7%95%B6%E6%88%91.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from sklearn import metrics
from PIL import Image
from torchvision import models, transforms
from torch.utils.data import Dataset, DataLoader
from tqdm.notebook import tqdm
from google.colab import drive

model_path = "model.pt"

##Connect Google Drive

In [16]:
drive.mount('/content/drive')
train_path = '/content/drive/My Drive/Deep_Learning/Midterm/train/'
os.chdir(train_path)
training_data_file = os.listdir(train_path)
print(training_data_file[0][:-11])
type(training_data_file[0])
# os.chdir(test_path)
# testing_data_file = os.listdir(test_path)
# print(len(training_data_file))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
WRIST_cfde832782


str

##Dataset

In [34]:
from torchvision.io import read_image, ImageReadMode
#train_dataset

Resize = transforms.Compose([
  transforms.Resize([512,512])
])


class BoneDataset(Dataset):
    def __init__(self, annotations_file, directory, transform=None, target_transform=None):
        self.dir = directory
        self.img_labels = pd.read_csv(annotations_file, index_col='id')
        # self.img_dir = [os.path.join(directory, f) for f in os.listdir(directory)]
        self.img_dir = os.listdir(directory)
        self.transform = transform
        self.target_transform = target_transform
        
        # for idx in range(4000):

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = self.img_dir[idx]
        image = read_image(os.path.join(self.dir, img_path), mode=ImageReadMode.RGB)
        image = image.type(torch.FloatTensor)
        # i=0
        # while self.img_labels.iloc[i, 0] != img_path[:-11]:
        #   i=i+1
        i = self.img_labels.loc[f"{img_path[:-11]}"]
        # print(i)
        # label = self.img_labels.iloc[i, 1]
        label = i['label']


        # Get patient ID
        # ID = self.img_labels.iloc[idx, 0]
        # patient_image = torch.tensor([])
        # i = 0
        # for f in os.listdir(self.dir):
        #   # If file name equals ID
        #   if patient_image.size(dim=0) == 4:
        #     break
        #   else:
        #     if f[:-11] == ID:
        #       img_path = self.img_dir[i]
        #       image = Resize(read_image(img_path, mode=ImageReadMode.GRAY))
        #       patient_image = torch.cat((patient_image, image), 0)
        #     i = i + 1
        # # print(f'patient{idx}: {patient_image.size(dim=0)}')
        # # print(patient_image.size(dim=0))
        # while patient_image.size(dim=0) < 4:
        #   if (4 - patient_image.size(dim=0)) == 1:
        #     image = transforms.functional.vflip(patient_image[2])
        #     torch.reshape(image, (1,image.shape[0], image.shape[1]))
        #     patient_image = torch.cat((patient_image, image), 0)
        #   elif (4 - patient_image.size(dim=0)) == 2:
        #     image = transforms.functional.hflip(patient_image[1])
        #     patient_image = torch.cat((patient_image, image), 0)
        #   else:
        #     image = transforms.functional.vflip(patient_image[0])
        #     patient_image = torch.cat((patient_image, image), 0)        
        # patient_image = patient_image.type(torch.FloatTensor)
        # # img_path = self.img_dir[idx]
        # # image = read_image(img_path, mode=ImageReadMode.RGB)
        # # image = image.type(torch.FloatTensor)
        # # label = self.img_labels.iloc[idx, 1]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label
        # label = self.img_labels.iloc[idx, 1]
        # if self.transform:
        #     patient_image = self.transform(patient_image)
        # if self.target_transform:
        #     label = self.target_transform(label)
        # return patient_image, label

In [31]:
train_path = '/content/drive/My Drive/Deep_Learning/Midterm/train/'
test_path = '/content/drive/My Drive/Deep_Learning/Midterm/test/'
label_file = '/content/drive/My Drive/Deep_Learning/Midterm/train.csv'

#model
batch_size = 32
learning_rate = 1e-4
num_show = 3
transform = transforms.Compose([
  transforms.Grayscale(num_output_channels=1),
  # transforms.PILToTensor(),
  transforms.Resize([512,512]),
  # transforms.ToPILImage(mode='RGB')
])



train_dataset = BoneDataset(label_file, train_path, transform=transform)
# test_dataset = ...


train_dataloader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle = True
)
'''
test_dataloader = torch.utils.data.DataLoader(
    #test_dataset,
    batch_size=1,
    shuffle = False
)
'''
model = models.efficientnet_b0(pretrained=True)
# model = models.resnet18(pretrained=True)
model.conv1 = nn.Conv2d(
    1,
    64,
    kernel_size=(7, 7),
    stride=(2, 2),
    padding=(3, 3),
    bias=False
)
model = nn.Sequential(model, nn.Linear(1000,1))
model = model.cuda()
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
loss_fn = nn.SmoothL1Loss()
# print(train_dataloader.dataset)

In [32]:
train_features, train_labels = next(iter(train_dataloader))
print(f"feature batch size: {train_features.size()}")
print(f"labels batch size: {train_labels.size()}")
img = train_features[1].squeeze()
label = train_labels[1]
print(img)
# plt.imshow(img)
# plt.show()
print(f"Label: {label}")

label   NaN
Name: WRIST_58f3d053c0, dtype: float64
label   NaN
Name: WRIST_aca5b1a42d, dtype: float64
label    1.0
Name: WRIST_ed0882dd08, dtype: float64
label   NaN
Name: WRIST_4ed41c5066, dtype: float64
label   NaN
Name: WRIST_5c31e4c2fe, dtype: float64
label   NaN
Name: WRIST_ae7241cb05, dtype: float64
label    1.0
Name: WRIST_f427cb2860, dtype: float64
label    0.0
Name: WRIST_5660e50893, dtype: float64
label    1.0
Name: WRIST_ff079757c2, dtype: float64
label   NaN
Name: WRIST_7eb64f6933, dtype: float64
label   NaN
Name: WRIST_9238280dad, dtype: float64
label    0.0
Name: WRIST_d5a5dd8a10, dtype: float64
label    1.0
Name: WRIST_d993e22745, dtype: float64
label   NaN
Name: WRIST_990800982d, dtype: float64
label   NaN
Name: WRIST_7a2577464f, dtype: float64
label    0.0
Name: WRIST_cda8c4665c, dtype: float64
label   NaN
Name: WRIST_8df98ab913, dtype: float64
label   NaN
Name: WRIST_7b60a7c53d, dtype: float64
label    1.0
Name: WRIST_e597206be3, dtype: float64
label    0.0
Name: WRIS

In [33]:
num_epochs = 1

#train the model
best_test_loss = np.inf

for idx_epoch in range(num_epochs):
  print(f"Epoch{idx_epoch}")

  #training phase
  model.train()

  train_losses = []
  for image_batch, label_batch in tqdm(train_dataloader):
    optimizer.zero_grad()
    image_batch = image_batch.cuda()
    label_batch = label_batch.cuda()
    output_batch = model(image_batch)
    # print(output_batch.shape)
    loss = loss_fn(output_batch, label_batch)
    loss.backward()
    optimizer.step()

    loss = loss.detach().cpu().numpy()
    train_losses.append(loss)

  train_loss = np.mean(train_losses)
  print(f"Training loss: {train_loss}")

  # y = np.array([1,1,2,2])
  # y_pred = np.array([0.1,0.4,0.35,2])
  # fpr, tpr, thresholds = metrics.roc_curve(y, y_pred, pos_label=2)
  # print(metrics.auc(fpr, tpr))

  '''
  #testing phase
  model.eval()

  test_losses = []
  test_accuracies = []
  for image_batch, label_batch in tqdm(test_dataloader):
    image_batch = image_batch.cuda()
    label_batch = label_batch.cuda()
    output_batch = model(image_batch)
    loss = loss_fn(output_batch, label_batch)
    
    loss = loss.detach().cpu().numpy()
    train_losses.append(loss)
    
    ##### AUC #####
    

    accuracy = np.mean(is_correct_batch)
    test_accuracies.append(accuracy)
    
    

    #####
    test_loss = np.mean(test_losses)
    test_accuracy = np.mean(test_accuracies)

    if test_loss < best_test_loss:
      best_test_loss = test_loss
      torch.save(model.state_dict(), model_path)
      print("The model is saved")
    '''
  torch.save(model.state_dict(), model_path)
  print("The model is saved")

Epoch0


  0%|          | 0/125 [00:00<?, ?it/s]

label    1.0
Name: WRIST_9fd42e54e3, dtype: float64
label    1.0
Name: WRIST_dd0c971990, dtype: float64
label   NaN
Name: WRIST_74b2d15b70, dtype: float64
label    0.0
Name: WRIST_e41eec7cb9, dtype: float64
label    0.0
Name: WRIST_7d090883e9, dtype: float64
label   NaN
Name: WRIST_af48682e04, dtype: float64
label   NaN
Name: WRIST_45e6d54e09, dtype: float64
label    0.0
Name: WRIST_b9461d5c34, dtype: float64
label    0.0
Name: WRIST_beac75f5c9, dtype: float64
label   NaN
Name: WRIST_7b1edfaf48, dtype: float64
label   NaN
Name: WRIST_77c4baba09, dtype: float64
label   NaN
Name: WRIST_6d30a6adc2, dtype: float64
label   NaN
Name: WRIST_9a19c8f5bd, dtype: float64
label   NaN
Name: WRIST_63c64799bd, dtype: float64


KeyboardInterrupt: ignored

#Playground

##Import CSV

In [29]:
df = pd.read_csv (r'/content/drive/My Drive/Deep_Learning/Midterm/train.csv',dtype=str,index_col='id')
i = df.loc["FOREARM_00f72d1153"]
print(i['label'])
sorted_df = df.sort_values(by='label',kind='mergesort')
# print(df)
# print(sorted_df)

0


##Import Data

In [10]:
train_dataset = torch.tensor([])
transform1 = transforms.Compose([
  transforms.PILToTensor(),
])
print(train_dataset)
for i in range(100):
  img = Image.open(f"{train_path}/{training_data_file[i]}")
  img_tensor = transform1(img)
  a = torch.cat((train_dataset, img_tensor),0)
  # print(a.shape)

tensor([])


KeyboardInterrupt: ignored